In [1337]:
#------- EXERCICI 1 --- ACABAT
import pandas as pd

data = pd.read_csv('DelayedFlights.csv')

df = data.sample(n=200000, replace=False)

df = df[[
    #'Unnamed: 0', 'Year', 'Month', 'DayofMonth',
    'DayOfWeek',
    #'DepTime',
    #'CRSDepTime', 'ArrTime', 'CRSArrTime',
    'UniqueCarrier', 'FlightNum', 'TailNum', 'ActualElapsedTime',
    #'CRSElapsedTime',
    'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'Distance', 
    #'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
    #'WeatherDelay', 'NASDelay', 'SecurityDelay',
    #'LateAircraftDelay'
    ]].copy()

En este dataset podemos encontrar información sobre los vuelos de 20 aerolíneas comerciales en los Estados Unidos en 2008. Ésta muestra del dataset contiene 30 columnas con datos sobre 200.000 vuelos tanto antes de realizarse, así como tras la finalización de los mismos. De esta manera, por ejemplo, podemos obtener información sobre retrasos, desviaciones o cancelaciones y sus causas.

Puesto que no sabría extraer información de ello, podría eliminar las dos columnas que hacen referencia a la bajada y la subida del tren de aterrizaje (TaxiIn y TaxiOut).

Por otro lado, tambien puedo eliminar todas las columnas que se refieren a los retrasos y las cancelaciones según la causa, así como las columnas sobre la fecha como "Year", "Month" o "DayOfMonth", como también aquellas relacionadas con la programación original del vuelo como "CRSDepTime" o "CRSElapsedTime", ya que tampoco haré uso de estas columnas para los ejercicios que siguen.

Finalmente, modifico el nombre de las columnas para facilitar la lectura y vuelvo a indexar los registros para darles un valor entre 0 y 200.000 acorde con la totalidad de la muestra y no con la base de datos original.

In [1338]:
df = df.rename(columns = {'DayOfWeek':'Day_Of_Week',
                          'UniqueCarrier':'Carrier',
                          'FlightNum':'Flight_Number',
                          'TailNum':'Tail_Number',
                          'ActualElapsedTime':'Actual_Elapsed_Time',
                          'AirTime':'Air_Time',
                          'ArrDelay':'Arrival_Delay',
                          'DepDelay':'Departure_Delay',
                          'Dest':'Destination'}) \
    .reset_index(drop=True)

In [1339]:
df.describe()

,Day_Of_Week,Flight_Number,Actual_Elapsed_Time,Air_Time,Arrival_Delay,Departure_Delay,Distance
count,200000.000000,200000.000000,199096.000000,199096.000000,199096.000000,200000.000000,200000.000000
mean,3.986305,2191.689880,133.315220,108.245771,42.416166,43.374565,765.654645
std,1.996957,1950.254876,72.114396,68.632967,57.474769,54.126721,574.638940
min,1.000000,1.000000,14.000000,1.000000,-109.000000,6.000000,11.000000
25%,2.000000,613.000000,80.000000,58.000000,9.000000,12.000000,340.000000
50%,4.000000,1548.000000,116.000000,90.000000,24.000000,24.000000,606.000000
75%,6.000000,3446.000000,165.000000,137.000000,56.000000,53.000000,998.000000
max,7.000000,9742.000000,790.000000,655.000000,1707.000000,1710.000000,4962.000000


Al ejecutar la función describe() podemos ver que faltan algunos datos en al menos 3 columnas, por lo que tendremos que volver a ellas más adelante.

Por otro lado, también podemos observar que el valor mínimo de la columna 'Arrival_Delay' es negativo. Esto es normal, ya que en esta misma columna se expresa mediante un valor negativo cuando el vuelo llega antes de lo programado.

Sin embargo, el valor máximo de la misma columna de 'Arrival_Delay' parece demasiado elevado en comparación con el 75% más bajo, que a penas llega a la hora de retraso, así que acto seguido miraremos si sigue una progresión normal.

Finalmente, también podemos observar que el valor mínimo de la columna 'Distance' parece muy bajo para un trayecto en avión, por lo que podría ser un error a primera vista, sin emabrgo al ver la cantidad de vuelos por distancia vemos que es normal que haya trayectos así de cortos.

In [1340]:
df_more_delayed = df.loc[df['Distance'] < 50]
df_more_delayed.sort_values(by='Distance', ascending=True)
#df['Distance'].plot(kind='hist', bins=100)

,Day_Of_Week,Carrier,Flight_Number,Tail_Number,Actual_Elapsed_Time,Air_Time,Arrival_Delay,Departure_Delay,Origin,Destination,Distance
180500,7,OH,5572,N819CA,60.0,14.0,55.0,55.0,JFK,LGA,11
165155,1,OH,6898,N710CA,133.0,27.0,88.0,15.0,HPN,JFK,30
36481,5,OH,5052,N442CA,74.0,23.0,22.0,8.0,HPN,JFK,30
247,3,AS,65,N771AS,17.0,8.0,6.0,14.0,WRG,PSG,31
70294,2,AS,64,N765AS,NaN,NaN,NaN,51.0,PSG,WRG,31
...,...,...,...,...,...,...,...,...,...,...,...
131124,2,OO,5457,N564SW,39.0,14.0,27.0,29.0,LAX,OXR,49
133558,7,OO,5457,N569SW,28.0,14.0,44.0,53.0,LAX,OXR,49
135821,7,OO,5514,N226SW,35.0,14.0,14.0,14.0,LAX,OXR,49
142040,4,OO,5512,N584SW,33.0,15.0,11.0,19.0,OXR,LAX,49


De vuelta al valor máximo de la misma columna de 'Arrival_Delay', al consultar los registros según el mayor tiempo de retraso podemos observar que parece disminuir de manera normal y que son pocos los vuelos que acumulan mucho retraso.

In [1359]:
df_more_delayed = df.loc[df['Arrival_Delay'] > 1000]
df_more_delayed.sort_values(by='Arrival_Delay', ascending=False)[:]
#df['Arrival_Delay'].plot(kind='hist', bins=300)

,Day_Of_Week,Carrier,Flight_Number,Tail_Number,Actual_Elapsed_Time,Air_Time,Arrival_Delay,Departure_Delay,Origin,Destination,Distance
129858,5,MQ,3538,N512AE,72.0,50.0,1707.0,1710.0,LIT,DFW,304
190350,1,NW,512,N322US,102.0,75.0,1583.0,1552.0,OMA,MSP,282
198109,6,AA,2093,N3CDAA,211.0,177.0,1370.0,1521.0,LGA,MIA,1097
111756,3,AA,1274,N3ASAA,107.0,86.0,1357.0,1355.0,DEN,DFW,641
4990,1,NW,776,N501US,175.0,159.0,1344.0,1358.0,LAS,MSP,1300
103231,7,NW,491,N521US,168.0,149.0,1263.0,1258.0,TPA,DTW,983
166970,7,NW,776,N534US,173.0,140.0,1190.0,1206.0,LAS,MSP,1300
62242,7,NW,1788,N9338,195.0,157.0,1166.0,1123.0,GRR,MCO,1023
181714,2,NW,1639,N555NW,160.0,104.0,1155.0,1141.0,EGE,MSP,788
71000,7,MQ,3744,N515AE,67.0,50.0,1148.0,1146.0,LBB,DFW,282


In [1342]:
df.dtypes

Day_Of_Week              int64
Carrier                 object
Flight_Number            int64
Tail_Number             object
Actual_Elapsed_Time    float64
Air_Time               float64
Arrival_Delay          float64
Departure_Delay        float64
Origin                  object
Destination             object
Distance                 int64
dtype: object

Por último vemos que el tipo de dato tiene sentido con lo que representan, así que los dejaremos sin modificar.

In [1343]:
#------- EXERCICI 2 --- ACABAT
### Troba quantes dades faltants hi ha per columna.
print('Los registros que faltan por columna son:')
pd.isnull(df).sum()

Los registros que faltan por columna son:


Day_Of_Week              0
Carrier                  0
Flight_Number            0
Tail_Number              1
Actual_Elapsed_Time    904
Air_Time               904
Arrival_Delay          904
Departure_Delay          0
Origin                   0
Destination              0
Distance                 0
dtype: int64

In [1344]:
#------- EXERCICI 2 --- ACABAT
### Depuración: Rellenar Na con la media.
df[['Actual_Elapsed_Time', 'Air_Time', 'Arrival_Delay']] = df[['Actual_Elapsed_Time', 'Air_Time', 'Arrival_Delay']].fillna(df[['Actual_Elapsed_Time', 'Air_Time', 'Arrival_Delay']].mean())

print(df.isna().sum())

Day_Of_Week            0
Carrier                0
Flight_Number          0
Tail_Number            1
Actual_Elapsed_Time    0
Air_Time               0
Arrival_Delay          0
Departure_Delay        0
Origin                 0
Destination            0
Distance               0
dtype: int64


In [1345]:
#------- EXERCICI 2 --- ACABAT
### Haz un análisis estadístico que consideres relevante.
df1 = df[['Day_Of_Week', 'Arrival_Delay']]
df1_DayOfWeek_Delay = df1.groupby('Day_Of_Week').aggregate(sum)
df1_DayOfWeek_Delay.sort_values(by='Arrival_Delay', ascending=False)

,Arrival_Delay
Day_Of_Week,
5,1.466864e+06
7,1.351093e+06
1,1.256419e+06
4,1.222772e+06
2,1.193885e+06
3,1.092033e+06
6,9.001671e+05


En esta nuevo data frame podemos ver que el mayor número de retrasos suceden los viernes (5) y los domingos (7) posiblemente debido a las idas y las vueltas del fin de semana, y que por la misma razón que el menor número de retrasos se dan en sábado (6).

In [1346]:
#------- EXERCICI 2 --- ACABAT
### Crear columna velocidad mediana de los vuelos, si han llegado tarde o no...
df2 = df[['Flight_Number', 'Air_Time', 'Distance', 'Arrival_Delay', 'Departure_Delay']]
df2["Mean_Speed(m/m)"] = df2['Distance'] / df2['Air_Time']
df2['Delayed'] = df2['Arrival_Delay'].apply(lambda x:'Yes' if x > 0 else 'No')
df2.head()

C:\Users\dcuxa\AppData\Local\Temp\ipykernel_32632\2382179659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Mean_Speed(m/m)"] = df2['Distance'] / df2['Air_Time']
C:\Users\dcuxa\AppData\Local\Temp\ipykernel_32632\2382179659.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Delayed'] = df2['Arrival_Delay'].apply(lambda x:'Yes' if x > 0 else 'No')


,Flight_Number,Air_Time,Distance,Arrival_Delay,Departure_Delay,Mean_Speed(m/m),Delayed
0,2721,99.0,717,29.0,32.0,7.242424,Yes
1,662,143.0,1011,202.0,200.0,7.069930,Yes
2,35,139.0,861,98.0,36.0,6.194245,Yes
3,406,79.0,550,14.0,32.0,6.962025,Yes
4,2780,148.0,1201,55.0,48.0,8.114865,Yes


In [1347]:
#------- EXERCICI 2 --- ACABAT
### Fes una taula de les aerolínies amb més endarreriments acumulats.
df3 = df[['Carrier', 'Arrival_Delay']]
df3_UniqueCarrier_ArrDelay = df3.groupby('Carrier').aggregate(sum)
df3_UniqueCarrier_ArrDelay.sort_values(by='Arrival_Delay', ascending=False)

,Arrival_Delay
Carrier,
WN,1.177255e+06
AA,9.147094e+05
UA,6.984257e+05
MQ,6.833757e+05
OO,6.281900e+05
XE,5.408648e+05
DL,4.781714e+05
CO,4.187156e+05
EV,4.084672e+05


In [1348]:
#------- EXERCICI 2 --- ACABAT
### ¿Cuáles son los vuelos más largos?
df4 = df[['Flight_Number', 'Air_Time']]
df4_more_airtime = df4.groupby('Flight_Number').aggregate('max')
df4_more_airtime.sort_values(by='Air_Time', ascending=False)[:5]

,Air_Time
Flight_Number,
15,655.0
1273,594.0
851,586.0
14,578.0
1561,551.0


In [1349]:
#------- EXERCICI 2 --- ACABAT
### I els més endarrerits?
df5 = df[['Flight_Number', 'Arrival_Delay']]
df5_more_delayed = df5.groupby('Flight_Number').aggregate('max')
df5_more_delayed.sort_values(by='Arrival_Delay', ascending=False)[:5]

,Arrival_Delay
Flight_Number,
3538,1707.0
512,1583.0
2093,1370.0
1274,1357.0
776,1344.0


In [1350]:
#------- EXERCICI 2 --- ACABAT
### Busca les rutes mes llarges.
df6 = df[['Distance', 'Origin', 'Destination']]
df6_longest_routes = df6.groupby(['Origin', 'Destination']).aggregate('max')
df6_longest_routes.sort_values(by=['Distance'], ascending=False)[:5]

Distance
Origin Destination          
EWR    HNL              4962
HNL    EWR              4962
       ATL              4502
ATL    HNL              4502
HNL    ORD              4243

In [1367]:
#------- EXERCICI 2 --- ACABAT
### ... y las rutas que acumulan más retrasos.
df7 = df[['Arrival_Delay', 'Origin', 'Destination']]
df7_more_delayed_routes = df7.groupby(['Origin', 'Destination']).aggregate(sum)
df7_more_delayed_routes.sort_values(by=['Arrival_Delay'], ascending=False)[:5]

,,Arrival_Delay
Origin,Destination,
LGA,ORD,25459.832332
ORD,LGA,25017.080830
LAX,SFO,23209.832332
ORD,EWR,21486.832332
EWR,ORD,20944.416166


In [1373]:
#------- EXERCICI 2 --- ACABAT
### Aporta aquello que consideres relevante.
df['Total_Flights'] = df['Tail_Number'].value_counts().reset_index(drop=True)
df8 = df[['Tail_Number', 'Arrival_Delay', 'Departure_Delay', 'Air_Time', 'Total_Flights']]
df8_planes_more_delayed = df8.groupby('Tail_Number').aggregate(sum)
df8_planes_more_delayed.sort_values(by='Arrival_Delay', ascending=False)[:10]

,Arrival_Delay,Departure_Delay,Air_Time,Total_Flights
Tail_Number,,,,
N17175,7468.000000,7010.0,4658.000000,36.0
N75998,6903.000000,6583.0,5146.000000,120.0
N154SF,6874.416166,7171.0,5013.245771,111.0
N651ML,6521.000000,6647.0,4458.000000,121.0
N571ML,6492.000000,6463.0,5616.000000,126.0
N592ML,6286.416166,6298.0,5507.245771,54.0
N77331,6237.000000,6258.0,4148.000000,80.0
N75992,5799.416166,5762.0,4397.245771,169.0
N27185,5770.000000,5831.0,5013.000000,0.0


En esta última consulta quería ver si había algún tipo de relación entre el avión que realiza el vuelo (Tail_Number), el retraso total y el número de vuelos realizado.

Vistos los resultados, puede que la base de datos describa una relación. Un siguiente paso podría ser el de investigar la causa de los retrasos de cada uno de los aviones ('Tail_Numnber') de esta última lista.

In [1372]:
df['Total_Flights'].max()

111.0

In [1353]:
#df.query("Tail_Number == 'N75992'")
df['Tail_Number'].value_counts()

N660SW    111
N382SW    108
N347SW    108
N641SW    107
N388SW    106
         ... 
N271AY      1
N1611B      1
N277AY      1
N7CCAA      1
N187DN      1
Name: Tail_Number, Length: 5256, dtype: int64

In [1354]:
#------- EXERCICI 3 --- ACABAT

#df.to_excel('Sprint_03_Tasca_02.xlsx', sheet_name='Sheet1')